## Phase 5 — Chain-of-Thought Judge with Self-Consistency

Advanced judging with reasoning chains and ensemble voting


In [16]:
import os
import json
import statistics
from collections import Counter
from openai import OpenAI
from typing import Dict, List, Optional
from dataclasses import dataclass


client = OpenAI(api_key=os.environ["OPENAI_API_KEY"])

MODEL_ANSWER = "gpt-4o-mini"   # responder
MODEL_JUDGE  = "gpt-4.1"        # judge


In [35]:
class ChainOfThoughtJudge:
    """Advanced judge using CoT reasoning and ensemble methods"""
    
    def __init__(self, num_judges: int = 3):
        self.num_judges = num_judges
    
    def generate_answer(self, question: str) -> str:
        """Generate answer"""
        message = client.chat.completions.create(
            model=MODEL_ANSWER,
            max_tokens=1000,
            messages=[
                {"role": "user", "content": question}
            ]
        )
        return message.choices[0].message.content
    
    def judge_with_cot(self, question: str, answer: str, judge_id: int = 1) -> Dict:
        """Single judge evaluation with chain-of-thought reasoning"""
        
        judge_prompt = f"""You are Judge #{judge_id} evaluating an answer. Use step-by-step reasoning.

Question: {question}

Answer to Evaluate:
{answer}

Think through this evaluation step-by-step:

1. COMPREHENSION: Does the answer demonstrate understanding of the question?
2. ACCURACY: Is the information factually correct?
3. COMPLETENESS: Are all aspects addressed?
4. CLARITY: Is it well-explained?
5. REASONING: Is the logic sound?

For each step, provide your reasoning, then give a score.

Finally, provide an overall judgment.

Respond ONLY with valid JSON:
{{
    "judge_id": {judge_id},
    "chain_of_thought": {{
        "step1_comprehension": {{
            "reasoning": "your detailed reasoning",
            "score": <1-10>
        }},
        "step2_accuracy": {{
            "reasoning": "your detailed reasoning",
            "score": <1-10>
        }},
        "step3_completeness": {{
            "reasoning": "your detailed reasoning",
            "score": <1-10>
        }},
        "step4_clarity": {{
            "reasoning": "your detailed reasoning",
            "score": <1-10>
        }},
        "step5_reasoning": {{
            "reasoning": "your detailed reasoning",
            "score": <1-10>
        }}
    }},
    "overall_score": <1-10>,
    "confidence": <1-10>,
    "verdict": "excellent" or "good" or "acceptable" or "poor",
    "final_reasoning": "synthesis of all steps"
}}
"""

        message = client.chat.completions.create(
            model=MODEL_JUDGE,
            max_tokens=2000,
            temperature=0.7 + (judge_id * 0.1),  # vary temperature slightly per judge
            messages=[
                {"role": "user", "content": judge_prompt}
            ]
        )
        
        response_text = message.choices[0].message.content
        
        try:
            if "```json" in response_text:
                response_text = response_text.split("```json")[1].split("```")[0]
            elif "```" in response_text:
                response_text = response_text.split("```")[1].split("```")[0]
            
            return json.loads(response_text.strip())
        except json.JSONDecodeError:
            return {
                "error": "Failed to parse JSON",
                "raw_response": response_text,
                "judge_id": judge_id
            }
    
    def ensemble_evaluate(self, question: str, answer: str) -> Dict:
        """Get evaluations from multiple judges and aggregate"""
        
        print(f"\n🎭 Assembling panel of {self.num_judges} judges...")
        
        judge_evaluations = []
        
        for i in range(1, self.num_judges + 1):
            print(f"  Judge {i}: Deliberating...")
            evaluation = self.judge_with_cot(question, answer, judge_id=i)
            
            if "error" not in evaluation:
                judge_evaluations.append(evaluation)
            else:
                print(f"    ⚠️  Judge {i} had an error")
        
        if not judge_evaluations:
            return {"error": "All judges failed to evaluate"}
        
        # Aggregate results
        aggregated = self._aggregate_evaluations(judge_evaluations)
        aggregated['individual_judges'] = judge_evaluations
        
        return aggregated
    
    def _aggregate_evaluations(self, evaluations: List[Dict]) -> Dict:
        """Aggregate multiple judge evaluations using various methods"""
        
        # Collect all scores
        all_scores = [e['overall_score'] for e in evaluations]
        confidences = [e['confidence'] for e in evaluations]
        verdicts = [e['verdict'] for e in evaluations]
        
        # Step scores
        step_scores = {
            'comprehension': [],
            'accuracy': [],
            'completeness': [],
            'clarity': [],
            'reasoning': []
        }
        
        for eval in evaluations:
            cot = eval['chain_of_thought']
            step_scores['comprehension'].append(cot['step1_comprehension']['score'])
            step_scores['accuracy'].append(cot['step2_accuracy']['score'])
            step_scores['completeness'].append(cot['step3_completeness']['score'])
            step_scores['clarity'].append(cot['step4_clarity']['score'])
            step_scores['reasoning'].append(cot['step5_reasoning']['score'])
        
        # Calculate aggregations
        verdict_counts = Counter(verdicts)
        majority_verdict = verdict_counts.most_common(1)[0][0]
        
        return {
            "num_judges": len(evaluations),
            "aggregated_scores": {
                "mean": statistics.mean(all_scores),
                "median": statistics.median(all_scores),
                "stdev": statistics.stdev(all_scores) if len(all_scores) > 1 else 0,
                "min": min(all_scores),
                "max": max(all_scores)
            },
            "step_scores_avg": {
                step: statistics.mean(scores) 
                for step, scores in step_scores.items()
            },
            "confidence": {
                "mean": statistics.mean(confidences),
                "min": min(confidences),
                "max": max(confidences)
            },
            "verdict": {
                "majority": majority_verdict,
                "distribution": dict(verdict_counts),
                "agreement_rate": verdict_counts[majority_verdict] / len(verdicts)
            },
            "consensus_strength": self._calculate_consensus(all_scores)
        }
    
    def _calculate_consensus(self, scores: List[float]) -> str:
        """Determine strength of consensus among judges"""
        if len(scores) < 2:
            return "single_judge"
        
        stdev = statistics.stdev(scores)
        
        if stdev < 1.0:
            return "strong"
        elif stdev < 2.0:
            return "moderate"
        else:
            return "weak"
    
    def self_consistency_check(
        self, 
        question: str, 
        answer: str,
        iterations: int = 3
    ) -> Dict:
        """
        Self-consistency: Run same judge multiple times,
        check for consistency in scoring
        """
        
        print(f"\n🔄 Running self-consistency check ({iterations} iterations)...")
        
        evaluations = []
        for i in range(iterations):
            print(f"  Iteration {i+1}...")
            eval_result = self.judge_with_cot(question, answer, judge_id=i+1)
            if "error" not in eval_result:
                evaluations.append(eval_result)
        
        if not evaluations:
            return {"error": "All iterations failed"}
        
        # Check consistency
        scores = [e['overall_score'] for e in evaluations]
        verdicts = [e['verdict'] for e in evaluations]
        
        score_variance = statistics.variance(scores) if len(scores) > 1 else 0
        verdict_consistency = len(set(verdicts)) == 1
        
        return {
            "iterations": len(evaluations),
            "scores": scores,
            "score_stats": {
                "mean": statistics.mean(scores),
                "variance": score_variance,
                "range": max(scores) - min(scores)
            },
            "verdicts": verdicts,
            "verdict_consistent": verdict_consistency,
            "consistency_rating": "high" if score_variance < 1.0 else "moderate" if score_variance < 4.0 else "low",
            "detailed_evaluations": evaluations
        }

In [36]:
def print_cot_evaluation(evaluation: Dict):
    """Pretty print chain-of-thought evaluation"""
    if "error" in evaluation:
        print(f"❌ Error: {evaluation['error']}")
        return
    
    print(f"\n👨‍⚖️ JUDGE #{evaluation['judge_id']}")
    print("=" * 70)
    
    print("\n🧠 CHAIN OF THOUGHT REASONING:\n")
    
    cot = evaluation['chain_of_thought']
    steps = [
        ("1️⃣  COMPREHENSION", cot['step1_comprehension']),
        ("2️⃣  ACCURACY", cot['step2_accuracy']),
        ("3️⃣  COMPLETENESS", cot['step3_completeness']),
        ("4️⃣  CLARITY", cot['step4_clarity']),
        ("5️⃣  REASONING", cot['step5_reasoning'])
    ]
    
    for label, step in steps:
        score = step['score']
        bar = "█" * score + "░" * (10 - score)
        print(f"{label}")
        print(f"  Score: [{bar}] {score}/10")
        print(f"  Reasoning: {step['reasoning']}\n")
    
    print(f"🎯 FINAL JUDGMENT:")
    print(f"  Overall Score: {evaluation['overall_score']}/10")
    print(f"  Confidence: {evaluation['confidence']}/10")
    print(f"  Verdict: {evaluation['verdict'].upper()}")
    print(f"\n  Final Reasoning: {evaluation['final_reasoning']}")
    print("\n" + "=" * 70)

In [37]:
def print_ensemble_results(results: Dict):
    """Pretty print ensemble results"""
    if "error" in results:
        print(f"❌ Error: {results['error']}")
        return
    
    print("\n" + "=" * 70)
    print("🎭 ENSEMBLE EVALUATION RESULTS")
    print("=" * 70)
    
    print(f"\n👥 Panel Size: {results['num_judges']} judges")
    print(f"🤝 Consensus Strength: {results['consensus_strength'].upper()}")
    
    print(f"\n📊 AGGREGATED SCORES:")
    agg = results['aggregated_scores']
    print(f"  Mean:   {agg['mean']:.2f}/10")
    print(f"  Median: {agg['median']:.2f}/10")
    print(f"  StdDev: {agg['stdev']:.2f}")
    print(f"  Range:  {agg['min']:.2f} - {agg['max']:.2f}")
    
    print(f"\n📋 AVERAGE STEP SCORES:")
    for step, score in results['step_scores_avg'].items():
        bar = "█" * int(score) + "░" * (10 - int(score))
        print(f"  {step.capitalize():<15} [{bar}] {score:.2f}/10")
    
    print(f"\n🎯 VERDICT:")
    verdict = results['verdict']
    print(f"  Majority: {verdict['majority'].upper()}")
    print(f"  Agreement: {verdict['agreement_rate']*100:.1f}%")
    print(f"  Distribution: {verdict['distribution']}")
    
    conf = results['confidence']
    print(f"\n💪 CONFIDENCE:")
    print(f"  Mean: {conf['mean']:.2f}/10")
    print(f"  Range: {conf['min']:.2f} - {conf['max']:.2f}")
    
    print("\n" + "=" * 70)

In [38]:
def print_consistency_check(results: Dict):
    """Pretty print self-consistency results"""
    if "error" in results:
        print(f"❌ Error: {results['error']}")
        return
    
    print("\n" + "=" * 70)
    print("🔄 SELF-CONSISTENCY CHECK RESULTS")
    print("=" * 70)
    
    print(f"\n🔁 Iterations: {results['iterations']}")
    
    print(f"\n📊 SCORE STATISTICS:")
    stats = results['score_stats']
    print(f"  Scores: {results['scores']}")
    print(f"  Mean: {stats['mean']:.2f}")
    print(f"  Variance: {stats['variance']:.2f}")
    print(f"  Range: {stats['range']:.2f}")
    
    print(f"\n🎯 VERDICT CONSISTENCY:")
    print(f"  Verdicts: {results['verdicts']}")
    print(f"  All Same: {'✅ Yes' if results['verdict_consistent'] else '❌ No'}")
    
    rating = results['consistency_rating']
    emoji = {"high": "🟢", "moderate": "🟡", "low": "🔴"}
    print(f"\n{emoji[rating]} CONSISTENCY RATING: {rating.upper()}")
    
    print("\n" + "=" * 70)

In [39]:
print("=" * 70)
print("PHASE 5: CHAIN-OF-THOUGHT JUDGE WITH SELF-CONSISTENCY")
print("=" * 70)
    
judge = ChainOfThoughtJudge(num_judges=3)

question = "Explain how neural networks learn through backpropagation"

print(f"\n❓ Question: {question}\n")
# Generate answer
print("🤖 Generating answer...")
answer = judge.generate_answer(question)
print(f"\n💬 Answer:\n{answer}\n")
    
# Example 1: Single judge with CoT
print("\n" + "=" * 70)
print("EXAMPLE 1: Single Judge with Chain-of-Thought")
print("=" * 70)
    
single_eval = judge.judge_with_cot(question, answer, judge_id=1)
print_cot_evaluation(single_eval)
    
# Example 2: Ensemble evaluation
print("\n\n" + "=" * 70)
print("EXAMPLE 2: Ensemble Evaluation (Multiple Judges)")
print("=" * 70)
    
ensemble_results = judge.ensemble_evaluate(question, answer)
print_ensemble_results(ensemble_results)
    
# Show individual judge details
print("\n\n📋 INDIVIDUAL JUDGE DETAILS:")
for i, judge_eval in enumerate(ensemble_results.get('individual_judges', []), 1):
    print(f"\n{'─'*70}")
    print_cot_evaluation(judge_eval)
    
# Example 3: Self-consistency check
print("\n\n" + "=" * 70)
print("EXAMPLE 3: Self-Consistency Check")
print("=" * 70)
    
consistency = judge.self_consistency_check(question, answer, iterations=3)
print_consistency_check(consistency)
    
print("\n" + "=" * 70)
print("✅ Advanced evaluation complete!")
print("=" * 70)

PHASE 5: CHAIN-OF-THOUGHT JUDGE WITH SELF-CONSISTENCY

❓ Question: Explain how neural networks learn through backpropagation

🤖 Generating answer...

💬 Answer:
Neural networks learn through a process called backpropagation, which is a key part of the training process. Backpropagation is an algorithm used to minimize the error in predictions made by the network by adjusting the weights of the connections between neurons. Here’s a step-by-step explanation of how this process works:

### 1. **Forward Pass**:
   - In the forward pass, the input data is fed into the neural network. Each layer of neurons processes the inputs by applying a weighted sum followed by a non-linear activation function.
   - This continues through the hidden layers until the final output layer is reached, producing a predicted output.

### 2. **Calculating Loss**:
   - Once the neural network produces an output, this prediction is compared to the actual output (the target value) using a loss function (e.g., Mean Sq